In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

class EnsembleFeatureGenerator:
    """
    Classe pour générer des features d'ensemble en utilisant plusieurs modèles
    Implémente une approche de stacking pour améliorer les prédictions
    """
    def __init__(self, models):
        self.models = models
        self.preprocessors = []
        
    def fit_transform(self, X_train, y_train, X_valid):
        # Configuration de la validation croisée avec 5 plis
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        # Initialisation des arrays pour stocker les prédictions
        train_predictions = np.zeros((X_train.shape[0], len(self.models)))
        valid_predictions = np.zeros((X_valid.shape[0], len(self.models)))
        
        # Identification des features catégorielles et numériques
        categorical_features = [col for col in ['ocean_proximity'] if col in X_train.columns]
        numeric_features = [col for col in X_train.columns if col not in categorical_features]
        
        # Boucle sur chaque modèle
        for i, model in enumerate(self.models):
            print(f"Training model {i+1}/{len(self.models)}...")
            
            # Pipeline de prétraitement pour les variables numériques
            numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),  # Gestion des valeurs manquantes
                ('scaler', StandardScaler())  # Standardisation des données
            ])
            
            # Pipeline de prétraitement pour les variables catégorielles
            categorical_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Gestion des valeurs manquantes
                ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))  # Encodage one-hot
            ])
            
            # Créer le préprocesseur avec gestion des valeurs manquantes
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)
                ] if categorical_features else [
                    ('num', numeric_transformer, numeric_features)
                ])
            
            # Prédictions de validation croisée
            for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
                X_fold_train = X_train.iloc[train_idx].copy()
                y_fold_train = y_train.iloc[train_idx].copy()
                X_fold_val = X_train.iloc[val_idx].copy()
                
                X_fold_train_processed = preprocessor.fit_transform(X_fold_train)
                X_fold_val_processed = preprocessor.transform(X_fold_val)
                
                model.fit(X_fold_train_processed, y_fold_train)
                train_predictions[val_idx, i] = model.predict(X_fold_val_processed)
            
            # Entraînement final
            X_train_processed = preprocessor.fit_transform(X_train)
            X_valid_processed = preprocessor.transform(X_valid)
            
            model.fit(X_train_processed, y_train)
            valid_predictions[:, i] = model.predict(X_valid_processed)
            self.preprocessors.append(preprocessor)
        
        return train_predictions, valid_predictions, X_train_processed, X_valid_processed
    
    def transform_test(self, X_test):
        test_predictions = np.zeros((X_test.shape[0], len(self.models)))
        X_test_processed = None
        
        for i, (model, preprocessor) in enumerate(zip(self.models, self.preprocessors)):
            if i == 0:  # Garder le premier preprocessing pour les features originales
                X_test_processed = preprocessor.transform(X_test)
            test_processed = preprocessor.transform(X_test)
            test_predictions[:, i] = model.predict(test_processed)
            
        return test_predictions, X_test_processed

# Chargement des données
print("Chargement des données...")
train_data = pd.read_csv('./ynov-data/train_housing_train.csv')
valid_data = pd.read_csv('./ynov-data/train_housing_valid.csv')
test_data = pd.read_csv('./ynov-data/test_housing.csv')

# Vérification des colonnes
print("\nColonnes dans les données:")
print("Train:", train_data.columns.tolist())
print("Valid:", valid_data.columns.tolist())
print("Test:", test_data.columns.tolist())

# Séparer features et target
X_train = train_data.drop(['median_house_value', 'id'], axis=1)
y_train = train_data['median_house_value']
X_valid = valid_data.drop(['median_house_value', 'id', 'prediction'], axis=1)
y_valid = valid_data['median_house_value']
X_test = test_data.drop(['id'], axis=1)

# Vérification des valeurs manquantes
print("\nValeurs manquantes dans X_train:", X_train.isnull().sum().sum())
print("Valeurs manquantes dans X_valid:", X_valid.isnull().sum().sum())
print("Valeurs manquantes dans X_test:", X_test.isnull().sum().sum())

# Définir les modèles avec des paramètres plus robustes
models = [
    # Modèles linéaires régularisés
    Ridge(alpha=1.0, random_state=42),  # Régularisation L2
    Lasso(alpha=0.001, random_state=42),  # Régularisation L1
    ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42),  # Combinaison L1 et L2
    # Modèles ensemblistes
    RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),  # Forêt aléatoire
    GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)  # Gradient Boosting
]

try:
    # Phase 1: Génération des features d'ensemble
    print("\nGénération des features d'ensemble...")
    ensemble_generator = EnsembleFeatureGenerator(models)
    train_meta_features, valid_meta_features, X_train_processed, X_valid_processed = ensemble_generator.fit_transform(
        X_train, y_train, X_valid
    )

    # Phase 2: Combinaison des features originales et des meta-features
    print("Combinaison des features...")
    X_train_enhanced = np.hstack([X_train_processed, train_meta_features])
    X_valid_enhanced = np.hstack([X_valid_processed, valid_meta_features])

    # Phase 3: Entraînement du modèle final
    print("Entraînement du modèle final...")
    final_model = LinearRegression()
    final_model.fit(X_train_enhanced, y_train)

    # Phase 4: Évaluation du modèle
    valid_predictions = final_model.predict(X_valid_enhanced)
    valid_predictions = np.clip(valid_predictions, 0, None)  # Clip des valeurs négatives
    r2 = r2_score(y_valid, valid_predictions)
    rmse = np.sqrt(mean_squared_error(y_valid, valid_predictions))

    print(f"\nPerformances sur l'ensemble de validation:")
    print(f"R² score: {r2:.4f}")  # Coefficient de détermination
    print(f"RMSE: {rmse:.2f}")    # Erreur quadratique moyenne

    # Phase 5: Génération des prédictions finales
    print("\nGénération des prédictions de test...")
    test_meta_features, X_test_processed = ensemble_generator.transform_test(X_test)
    X_test_enhanced = np.hstack([X_test_processed, test_meta_features])
    test_predictions = final_model.predict(X_test_enhanced)
    test_predictions = np.clip(test_predictions, 0, None)

    # Phase 6: Création du fichier de soumission
    print("Création du fichier de soumission...")
    submission = pd.DataFrame({
        'id': test_data['id'], 
        'median_house_value': test_predictions
    })
    submission.to_csv('./ynov-data/submit.csv', index=False)
    print("\nTerminé! Fichier de soumission créé avec succès.")

except Exception as e:
    print(f"\nUne erreur s'est produite: {str(e)}")
    raise

Chargement des données...

Colonnes dans les données:
Train: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'id']
Valid: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'id', 'prediction']
Test: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity', 'id']

Valeurs manquantes dans X_train: 102
Valeurs manquantes dans X_valid: 21
Valeurs manquantes dans X_test: 84

Génération des features d'ensemble...
Training model 1/5...
Training model 2/5...


C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.895e+12, tolerance: 1.018e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.909e+12, tolerance: 1.011e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not conv

Training model 3/5...


C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.780e+13, tolerance: 1.018e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.766e+13, tolerance: 1.011e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not conv

Training model 4/5...
Training model 5/5...
Combinaison des features...
Entraînement du modèle final...

Performances sur l'ensemble de validation:
R² score: 0.8105
RMSE: 50956.15

Génération des prédictions de test...
Création du fichier de soumission...

Terminé! Fichier de soumission créé avec succès.
